In [281]:
import numpy as np
import pandas as pd
import json
import os
from PIL import Image

In [157]:
df = pd.read_csv('C:/Users/shvet/Downloads/train-all.csv')

In [158]:
df

,Unnamed: 0,INPUT:image,OUTPUT:path,rle
0,0,https://storage.yandexcloud.net/garbage-images...,"{""shape"":""polygon""\,""points"":[{""left"":0.406846...",1942571 3 1945269 9 1947967 15 1950665 21 1953...
1,1,https://storage.yandexcloud.net/garbage-images...,"{""shape"":""polygon""\,""points"":[{""left"":0.530583...",1504852 3 1507554 9 1510257 15 1512959 21 1515...
2,2,https://storage.yandexcloud.net/garbage-images...,"{""shape"":""polygon""\,""points"":[{""left"":0.120259...",3120742 2 3123444 6 3126147 10 3128850 13 3131...
3,3,https://storage.yandexcloud.net/garbage-images...,"{""shape"":""polygon""\,""points"":[{""left"":0.297034...",841684 2 844385 7 847086 12 849787 17 852488 2...
4,4,https://storage.yandexcloud.net/garbage-images...,"{""shape"":""polygon""\,""points"":[{""left"":0.719647...",36947 5 39641 10 42335 12 45030 13 47724 15 50...
...,...,...,...,...
4232,5242,https://storage.yandexcloud.net/garbage-images...,"{""shape"":""polygon""\,""points"":[{""left"":0.267891...",1942439 4 1945137 11 1947835 18 1950533 25 195...
4233,5244,https://storage.yandexcloud.net/garbage-images...,"{""shape"":""polygon""\,""points"":[{""left"":0.411109...",1047560 1 1050263 3 1052966 5 1055669 7 105837...
4234,5246,https://storage.yandexcloud.net/garbage-images...,"{""shape"":""polygon""\,""points"":[{""left"":0.389559...",2651023 19 2653720 42 2656418 49 2659115 58 26...
4235,5247,https://storage.yandexcloud.net/garbage-images...,"{""shape"":""polygon""\,""points"":[{""left"":0.565837...",1875487 1 1878189 3 1880891 6 1880933 2 188359...


In [159]:
df = df.rename(columns={'INPUT:image': 'image', 'OUTPUT:path': 'count'})

df = df[['image', 'count', 'rle']]
df

,image,count,rle
0,https://storage.yandexcloud.net/garbage-images...,"{""shape"":""polygon""\,""points"":[{""left"":0.406846...",1942571 3 1945269 9 1947967 15 1950665 21 1953...
1,https://storage.yandexcloud.net/garbage-images...,"{""shape"":""polygon""\,""points"":[{""left"":0.530583...",1504852 3 1507554 9 1510257 15 1512959 21 1515...
2,https://storage.yandexcloud.net/garbage-images...,"{""shape"":""polygon""\,""points"":[{""left"":0.120259...",3120742 2 3123444 6 3126147 10 3128850 13 3131...
3,https://storage.yandexcloud.net/garbage-images...,"{""shape"":""polygon""\,""points"":[{""left"":0.297034...",841684 2 844385 7 847086 12 849787 17 852488 2...
4,https://storage.yandexcloud.net/garbage-images...,"{""shape"":""polygon""\,""points"":[{""left"":0.719647...",36947 5 39641 10 42335 12 45030 13 47724 15 50...
...,...,...,...
4232,https://storage.yandexcloud.net/garbage-images...,"{""shape"":""polygon""\,""points"":[{""left"":0.267891...",1942439 4 1945137 11 1947835 18 1950533 25 195...
4233,https://storage.yandexcloud.net/garbage-images...,"{""shape"":""polygon""\,""points"":[{""left"":0.411109...",1047560 1 1050263 3 1052966 5 1055669 7 105837...
4234,https://storage.yandexcloud.net/garbage-images...,"{""shape"":""polygon""\,""points"":[{""left"":0.389559...",2651023 19 2653720 42 2656418 49 2659115 58 26...
4235,https://storage.yandexcloud.net/garbage-images...,"{""shape"":""polygon""\,""points"":[{""left"":0.565837...",1875487 1 1878189 3 1880891 6 1880933 2 188359...


In [161]:
df['count'] = df['count'].apply(lambda x: x.count('shape'))

AttributeError: 'int' object has no attribute 'count'

In [162]:
df['image'] = df['image'].apply(lambda x: x.split('/')[-1])

In [163]:
df = df.set_index('image')

In [229]:
df.loc['DJI_0037-400.jpg']

,count,rle
image,,
DJI_0037-400.jpg,1,1942571 3 1945269 9 1947967 15 1950665 21 1953...
DJI_0037-400.jpg,1,1961491 15 1964189 20 1966892 21 1969595 22 19...
DJI_0037-400.jpg,1,1929037 1 1931741 2 1934444 4 1937148 5 193985...


In [197]:
df.index.value_counts().value_counts()

2    625
3    427
1    384
4    150
5    115
6     14
7      9
Name: image, dtype: int64

In [248]:
def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

In [249]:
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [250]:
groups = df.groupby(df.index).size()
groups = groups[groups > 1]
groups

image
DJI_0008-0.jpg       2
DJI_0008-100.jpg     2
DJI_0008-1075.jpg    3
DJI_0008-1100.jpg    3
DJI_0008-1125.jpg    3
                    ..
DJI_0265-675.jpg     2
DJI_0265-725.jpg     2
DJI_0265-75.jpg      2
DJI_0265-750.jpg     2
DJI_0265-775.jpg     2
Length: 1340, dtype: int64

In [305]:
def replace_duplicates(rows):
    df.drop_duplicates(subset=df.index, keep=False, inplace=True)
    for row in rows:
        #row - tuple: 1 elem - string, 2 elem - [cnt, mask]
        df.loc[row[0]] = row[1]
        df.index = df.index + 1
        
def get_resolution(image_path: str):
    with Image.open('C:/Users/shvet/Documents/Machine Learning/ecodrone/train/train/'+image_path) as img:
        width, height = img.size
    return (height, width)
        
unified = []

for image in groups.index:
    dup = df.loc[image]
    height, width = get_resolution(image)
    final_mask = np.zeros(height * width, dtype=np.int32).reshape(-1, height)
    
    for i, rle in enumerate(dup['rle'], start=1):
        mask = rle_decode(rle, (width, height))
        np.add(final_mask, mask)
        
    #try:
    cur = rle_encode(final_mask.astype(np.int32))
    unified.append((image, [i, cur]))
    print(image)
    #except Exception as e:
    #    print(final_mask.shape)
    
    
replace_duplicates(unified)
    
    

C:\Users\shvet\Documents\Machine Learning


KeyError: Index(['image'], dtype='object')

In [319]:
def zip_duplicate_groups(groups):
    unified = []
        
    for image in groups.index:
        dup = df.loc[image]
        height, width = get_resolution(image)
        final_mask = np.zeros(height * width, dtype=np.int32).reshape(-1, height)

        for i, rle in enumerate(dup['rle'], start=1):
            mask = rle_decode(rle, (width, height))
            print(mask.astype(np.int32).reshape(1,-1).any(1))
            np.add(final_mask, mask)
        cur = rle_encode(final_mask.astype(np.int32))
        unified.append((image, [i, cur]))
        print(final_mask.astype(np.int32).reshape(1,-1).any(1))
        break
    
    return unified

In [320]:
unified = zip_duplicate_groups(groups)
print(unified)
replace_duplicates(unified)

[ True]
[ True]
[False]
[('DJI_0008-0.jpg', [2, ''])]


KeyError: Index(['image'], dtype='object')

In [231]:
rle_1 = '1942571 3 1945269 9 1947967 15 1950665 21 1953363 27 1956061 34 1958759 40 1961460 43 1964163 44 1966865 46 1969568 48 1972271 49 1974973 51 1977676 52 1980379 53 1983081 56 1985784 57 1988487 58 1991189 60 1993892 61 1996595 63 1999297 65 2002000 66 2004703 67 2007405 69 2010108 71 2012811 72 2015514 73 2018217 74 2020920 75 2023623 77 2026326 78 2029029 79 2031732 80 2034436 80 2037139 82 2039842 83 2042545 84 2045248 85 2047951 86 2050654 88 2053357 89 2056060 90 2058763 91 2061466 92 2064169 94 2066872 94 2069575 95 2072278 96 2074982 96 2077685 97 2080388 98 2083091 99 2085794 100 2088497 101 2091200 102 2093903 103 2096606 104 2099311 103 2102015 103 2104720 102 2107424 102 2110129 101 2112833 101 2115538 100 2118242 100 2120946 100 2123651 99 2126355 99 2129060 98 2131764 98 2134469 97 2137173 97 2139878 96 2142582 95 2145287 94 2147991 94 2150696 93 2153400 93 2156104 93 2158809 92 2161513 92 2164218 91 2166922 91 2169627 90 2172331 90 2175036 89 2177741 88 2180445 87 2183150 86 2185854 86 2188559 85 2191264 84 2193968 84 2196673 83 2199378 82 2202082 82 2204787 81 2207491 81 2210196 80 2212901 79 2215605 79 2218310 77 2221014 77 2223719 76 2226424 75 2229128 75 2231833 74 2234537 74 2237242 73 2239947 72 2242651 72 2245356 71 2248061 70 2250765 70 2253470 69 2256174 68 2258879 65 2261584 63 2264288 62 2266993 59 2269699 56 2272405 53 2275110 50 2277816 47 2280522 44 2283228 40 2285933 38 2288639 35 2291345 31 2294050 29 2296756 26 2299462 22 2302167 20 2304873 17 2307579 13 2310285 10 2312990 8 2315696 4 2318402 1'

In [301]:
height, width = get_resolution(rle_1)
mask1 = rle_decode(rle_1, (height, width))
rle2 = rle_encode(mask1.astype(np.int32))
mask1.shape


OSError: [Errno 22] Invalid argument: 'C:/Users/shvet/Documents/Machine Learning/ecodrone/train/train/1942571 3 1945269 9 1947967 15 1950665 21 1953363 27 1956061 34 1958759 40 1961460 43 1964163 44 1966865 46 1969568 48 1972271 49 1974973 51 1977676 52 1980379 53 1983081 56 1985784 57 1988487 58 1991189 60 1993892 61 1996595 63 1999297 65 2002000 66 2004703 67 2007405 69 2010108 71 2012811 72 2015514 73 2018217 74 2020920 75 2023623 77 2026326 78 2029029 79 2031732 80 2034436 80 2037139 82 2039842 83 2042545 84 2045248 85 2047951 86 2050654 88 2053357 89 2056060 90 2058763 91 2061466 92 2064169 94 2066872 94 2069575 95 2072278 96 2074982 96 2077685 97 2080388 98 2083091 99 2085794 100 2088497 101 2091200 102 2093903 103 2096606 104 2099311 103 2102015 103 2104720 102 2107424 102 2110129 101 2112833 101 2115538 100 2118242 100 2120946 100 2123651 99 2126355 99 2129060 98 2131764 98 2134469 97 2137173 97 2139878 96 2142582 95 2145287 94 2147991 94 2150696 93 2153400 93 2156104 93 215880

,image,rectangles
0,https://storage.yandexcloud.net/garbage-images...,"{""points"": [[0.3718361098715851, 0.41019884380..."
1,https://storage.yandexcloud.net/garbage-images...,"{""points"": [[0.4749768303985171, 0.59082483781..."
2,https://storage.yandexcloud.net/garbage-images...,"{""points"": [[0.10464685604026558, 0.1425639573..."
3,https://storage.yandexcloud.net/garbage-images...,"{""points"": [[0.2043558850787766, 0.30352177942..."
4,https://storage.yandexcloud.net/garbage-images...,"{""points"": [[0.6158480074142725, 0.74559777571..."
...,...,...
4232,https://storage.yandexcloud.net/garbage-images...,"{""points"": [[0.08904257961818024, 0.3883591555..."
4233,https://storage.yandexcloud.net/garbage-images...,"{""points"": [[0.3818553702463659, 0.41658197029..."
4234,https://storage.yandexcloud.net/garbage-images...,"{""points"": [[0.36358194325044296, 0.4306899703..."
4235,https://storage.yandexcloud.net/garbage-images...,"{""points"": [[0.5629642709420879, 0.63638235760..."


[[0.3718361098715851,
  0.4101988438082816,
  0.47252982038615804,
  0.5639652502146337]]

,rectangles
0,"{""points"": [[0.3718361098715851, 0.41019884380..."
1,"{""points"": [[0.4749768303985171, 0.59082483781..."
2,"{""points"": [[0.10464685604026558, 0.1425639573..."
3,"{""points"": [[0.2043558850787766, 0.30352177942..."
4,"{""points"": [[0.6158480074142725, 0.74559777571..."
...,...
4232,"{""points"": [[0.08904257961818024, 0.3883591555..."
4233,"{""points"": [[0.3818553702463659, 0.41658197029..."
4234,"{""points"": [[0.36358194325044296, 0.4306899703..."
4235,"{""points"": [[0.5629642709420879, 0.63638235760..."


<function __main__.<lambda>(x)>